In [1]:
# pip install flask

In [2]:
# importing required libraries
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [3]:
# read the dataset
data = pd.read_csv('twitter_sentiments.csv')
# view the top rows
data.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
# train test split
train, test = train_test_split(data, test_size = 0.2, stratify = data['label'], random_state=21)

# get the shape of train and test split.
train.shape, test.shape
## >> ((25569, 3), (6393, 3))

((25569, 3), (6393, 3))

In [5]:
# create a TF-IDF vectorizer object
tfidf_vectorizer = TfidfVectorizer(lowercase= True, max_features=1000, stop_words=ENGLISH_STOP_WORDS)

# fit the object with the training data tweets
tfidf_vectorizer.fit(train.tweet)

TfidfVectorizer(max_features=1000,
                stop_words=frozenset({'a', 'about', 'above', 'across', 'after',
                                      'afterwards', 'again', 'against', 'all',
                                      'almost', 'alone', 'along', 'already',
                                      'also', 'although', 'always', 'am',
                                      'among', 'amongst', 'amoungst', 'amount',
                                      'an', 'and', 'another', 'any', 'anyhow',
                                      'anyone', 'anything', 'anyway',
                                      'anywhere', ...}))

In [6]:
# transform the train and test data
train_idf = tfidf_vectorizer.transform(train.tweet)
test_idf  = tfidf_vectorizer.transform(test.tweet)

In [7]:
# create the object of LinearRegression Model
model_LR = LogisticRegression()

# fit the model with the training data
model_LR.fit(train_idf, train.label)

# predict the label on the traning data
predict_train = model_LR.predict(train_idf)

# predict the model on the test data
predict_test = model_LR.predict(test_idf)

# f1 score on train data
f1_score(y_true= train.label, y_pred= predict_train)
## >> 0.4888178913738019

f1_score(y_true= test.label, y_pred= predict_test)
## >> 0.45751633986928114

0.45751633986928114

In [8]:
# define the stages of the pipeline
pipeline = Pipeline(steps= [('tfidf', TfidfVectorizer(lowercase=True,
                                                      max_features=1000,
                                                      stop_words= ENGLISH_STOP_WORDS)),
                            ('model', LogisticRegression())])

# fit the pipeline model with the training data                            
pipeline.fit(train.tweet, train.label)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=1000,
                                 stop_words=frozenset({'a', 'about', 'above',
                                                       'across', 'after',
                                                       'afterwards', 'again',
                                                       'against', 'all',
                                                       'almost', 'alone',
                                                       'along', 'already',
                                                       'also', 'although',
                                                       'always', 'am', 'among',
                                                       'amongst', 'amoungst',
                                                       'amount', 'an', 'and',
                                                       'another', 'any',
                                                       'anyhow', 'anyone',
                                                       'anything', 'anyway',
                                                       'anywhere', ...}))),
                ('model', LogisticRegression())])

In [9]:
# sample tweet
text = ["Virat Kohli, AB de Villiers set to auction their 'Green Day' kits from 2016 IPL match to raise funds"]

# predict the label using the pipeline
pipeline.predict(text)
## >> array([0])

array([0], dtype=int64)

In [10]:
# import joblib
from joblib import dump

In [11]:
# dump the pipeline model
dump(pipeline, filename="text_classification.joblib")

['text_classification.joblib']

In [12]:
# import joblib
from joblib import load

# sample tweet text
text = ["Virat Kohli, AB de Villiers set to auction their 'Green Day' kits from 2016 IPL match to raise funds"]

# load the saved pipleine model
pipeline = load("text_classification.joblib")

# predict on the sample tweet text
pipeline.predict(text)
## >> array([0])

array([0], dtype=int64)

In [13]:
# !pip3 install tweepy

In [14]:
# import required libraries
import tweepy
import time
import pandas as pd
pd.set_option('display.max_colwidth', 1000)

# api key
api_key = "9D6LvvCirf5d16SudvkS2SiKf"
# api secret key
api_secret_key = "5RIuJlxMfsc3drxlwibWc5qgyf2rZPPb9ZPNcTsBFDKkTEx0gf"
# access token
access_token = "1577475918192201730-nzndXCXcumuxdZfj2DkGZTrXnIxGYT"
# access token secret
access_token_secret = "bzqvlVwjcKF4h13Rwcu35aX5JvxGHVUQQ1Z65TmdNDk2G"

# authorize the API Key
authentication = tweepy.OAuthHandler(api_key, api_secret_key)

# authorization to user's access token and access token secret
authentication.set_access_token(access_token, access_token_secret)

# call the api
api = tweepy.API(authentication, wait_on_rate_limit=True)

In [15]:
def get_related_tweets(text_query):
    # list to store tweets
    tweets_list = []
    # no of tweets
    count = 50
    try:
        # Pulling individual tweets from query
        for tweet in api.search_tweets(q=text_query, count=count):
            print(tweet.text)
            # Adding to list that contains all tweets
            tweets_list.append({'created_at': tweet.created_at,
                                'tweet_id': tweet.id,
                                'tweet_text': tweet.text})
        return pd.DataFrame.from_dict(tweets_list)

    except BaseException as e:
        print('failed on_status,', str(e))
        time.sleep(3)

In [16]:
from get_tweets import get_related_tweets

In [17]:
# importing the required libraries
from flask import Flask, render_template, request, redirect, url_for
from joblib import load


# load the pipeline object
pipeline = load("text_classification.joblib")

# function to get results for a particular text query
def requestResults(name):
    # get the tweets text
    tweets = get_related_tweets(name)
    # get the prediction
    tweets['prediction'] = pipeline.predict(tweets['tweet_text'])
    # get the value counts of different labels predicted
    data = str(tweets.prediction.value_counts()) + '\n\n'
    return data + str(tweets)

In [18]:
# start flask
app = Flask(__name__)

# render default webpage
@app.route('/')
def home():
    return render_template('home.html')

# when the post method detect, then redirect to success function
@app.route('/', methods=['POST', 'GET'])
def get_data():
    if request.method == 'POST':
        user = request.form['search']
        return redirect(url_for('success', name=user))

# get the data for the requested query
@app.route('/success/<name>')
def success(name):
    return "<xmp>" + str(requestResults(name)) + " </xmp> "

In [ ]:
app.run(debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Dec/2022 19:45:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 19:45:20] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Dec/2022 19:45:28] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [18/Dec/2022 19:45:29] "GET /success/elon%20musk HTTP/1.1" 200 -


RT @MattWallace888: Do you support the way Elon Musk is running Twitter?
Elon musk, the man, is a human shitshow. What a fucking maroon. https://t.co/OmAmuiWPn5
RT @larryelder: If Adolph Hitler, Mao Tse Tung and Elon Musk were walking down the street, and you gave an American lefty a gun with two bu…
RT @MattWallace888: Elon Musk just ended Anthony Fauci’s entire career 😱
RT @RBReich: Do I have this right…conservatives love free speech for Elon Musk, but hate free speech for Colin Kaepernick?
Elon Musk seeks additional funds for Twitter

https://t.co/YsHxn3JRrk
RT @MarlowNYC: the last tweet @taylorlorenz sent before elon musk kicked her off twitter: asking for comment for a story https://t.co/DYAjl…
RT @AntonioCasilli: Pendant que vous piquez une crise sur la décision de Musk d'accorder un pouvoir de downvote aux "nouveaux vérifiés" (=c…
RT @FordFischer: Elon Musk has apparently banned Washington Post’s @TaylorLorenz from the platform. https://t.co/M6f6l1txyq
ELON MUSK if u see this tw

127.0.0.1 - - [18/Dec/2022 19:45:29] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Dec/2022 19:45:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 19:46:03] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [18/Dec/2022 19:46:03] "GET /success/trump HTTP/1.1" 200 -


RT @LarryMarcey: Isn’t it amazing how the only people that get investigated in America is Trump, his family, business associates, his suppo…
@LesleyRStahl @OnPointRadio Forget the grandparenting. How about telling Trump the crap that Obama was doing was un… https://t.co/9uLcVlaRLn
@coldkappishake I can not let Trump outl1ve me
@Brrrbon_ Wow your comparative looks great, thanks for it!  Should Trump launch in Cardano? 🤔
RT @zsk: Taylor Lorenz seems to be the latest journalist suspended from Twitter. 
On Friday I wrote that Musk's definition of what's allowe…
@oskarzwittau @RonFilipkowski Trump going to prison is in fact the fair outcome that most countries feel he deserve… https://t.co/kNWqAZt4bx
@DanFarfan @DavidSacks @elonmusk Does the fact that there is like really obvious and open evidence that Trump did c… https://t.co/IhFXypvQTT
RT @AWeissmann_: What will the House Rs do with the J6C data dump? Try to obstruct as much as possible. Don't be surprised to see them in 2…
@RobertM78455

127.0.0.1 - - [18/Dec/2022 19:47:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 19:47:49] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [18/Dec/2022 19:47:50] "GET /success/trump HTTP/1.1" 200 -


@nSEnishant Agar abhi b ban hain tho Fir Duplicate modi gye te kya Donald Trump se milne k liye???? 😄
@RepMTG Bogus? It's been well over a year of exhaustive investigations with a literally mountain of evidence.  Ever… https://t.co/D1jFT82Amx
RT @ChanceGardiner: Wikipedia crea la voce "Il massacro del giovedì sera" per la sospensione momentanea di una decina di giornalisti Dem ch…
@EmilioRose9 ill be collecting my winnings from the bookies, i have a bet since 2017 that trump would go to jail
@MadScientistFF Trump
RT @duty2warn: All 45,000 Trump cards are sold. This week Trump made 4.5 million dollars selling stolen images that aren’t NFTs but are car…
RT @thehill: "Is DeSantis about to leverage COVID vaccines against Trump and the Democrats?" (@TheHillOpinion) https://t.co/YW7NifwSKl http…
RT @JRNYcrypto: Trump NFTs up 6x since mint 🤯
RT @hilarysontag: #January6thInsurrection #January6thHearing #DOJ #TrumpCoupAttempt #DonaldTrump 👇
EVERY SINGLE GOP lawmaker who texted Mar…
RT @slh06194